### Download videos from TikTok using YTL DLP

This script uses the library [`YTL DLP`](https://github.com/yt-dlp/yt-dlp) to download TikTok videos.

In [1]:
import json
import time

import pandas as pd
import yt_dlp

This sets up a function to download videos:

In [2]:
# ℹ️ See help(yt_dlp.YoutubeDL) for a list of available options and public functions

foldername = "demure" #<-- specify that name of the folder in your output folder where you want to store videos
ydl_opts = {
    'quiet': True,
    'no_warnings': True,
    'skip_download': False, 
    'outtmpl': '../output/'+ foldername+'/%(id)s.%(ext)s' #<-- this determines the file name and location
}

ydl = yt_dlp.YoutubeDL(ydl_opts)

def get_video_metadata(video_url):
    try:
        info = ydl.extract_info(video_url, download=True)
        return info
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

Deprecated Feature: Support for Python version 3.8 has been deprecated. Please update to Python 3.9 or above


#### Import urls
Here we need to import links from a spreadsheet for us to download from TikTok. 

In [3]:
videos = pd.read_csv("../output/@joolieannietiktoks.csv")
videos.head()

,link,video_count,handle
0,https://www.tiktok.com/@joolieannie/video/7399...,53.3M,@joolieannie
1,https://www.tiktok.com/@joolieannie/video/7425...,11.9K,@joolieannie
2,https://www.tiktok.com/@joolieannie/video/7425...,52.7K,@joolieannie
3,https://www.tiktok.com/@joolieannie/video/7425...,107K,@joolieannie
4,https://www.tiktok.com/@joolieannie/video/7425...,335.5K,@joolieannie


In [4]:
video_urls = videos["link"].tolist()
print(len(video_urls))

921


The next few lines take the links and download each video. Note that in this script we're only getting the first ten videos. If you want to download all videos from the profile, change this line `for video_url in video_urls[:10]:` to `for video_url in video_urls:`.

In [5]:
%%time 
rows = []

for video_url in video_urls[:10]: # <-- note that in this script we're only getting the first ten videos 

    metadata = get_video_metadata(video_url)
    if metadata:
        # Print the entire metadata dictionary
        # print(json.dumps(metadata, indent=2))
        row = {
            "title": metadata['title'] if 'title' in metadata else None,
            "duration": metadata['duration'] if 'duration' in metadata else None,
            "view_count": metadata['view_count'] if 'view_count' in metadata  else None,
            "upload_date": metadata['upload_date'] if 'upload_date' in metadata  else None,
            "artist": metadata['artist'] if 'artist' in metadata else None,
            "like_count": metadata["like_count"] if 'like_count' in metadata  else None,
            "repost_count": metadata["repost_count"] if 'repost_count' in metadata else None,
            "comment_count": metadata["comment_count"] if 'comment_count' in metadata else None,
            "video_url": video_url
        }
        rows.append(row)

CPU times: user 1.37 s, sys: 720 ms, total: 2.09 s         
Wall time: 27.4 s


Then we load the videos as a dataframe and export it as a csv file: 

In [6]:
videos_metadata = pd.DataFrame(rows)

videos_metadata.to_csv("../output/tiktok_videos_metadata.csv", index=False)

In [7]:
len(rows)


10